In [2]:
import os
import re
import json

from typing import Dict, List

def load_jsonl(file):
    with open(file, "r") as f:
        data = []
        for line in f:
            data.append(json.loads(line))
    return data

In [3]:
file_path = r"../infer/output/deepseek/generated_predictions_deepseek3_0.jsonl"

data = load_jsonl(file_path)

In [4]:
prompt_template = """
你是资深的产业专家！请先浏览下述产业环节的描述文本。
{name} {node}产业链环节：{desc}
接下来，请你逐一浏览行业分类列表：{selected_elements}，并在其中找出一眼看上去就属于上述产业链环节的行业，不需要做过渡的行业内容挖掘。
以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]
""".strip()

In [5]:
excepts = []
for item in data:
    try:
        assert "</think>" in item["predict"]
    except:
        excepts.append(item)

In [6]:
len(excepts)

0

In [7]:
data[0]

{'prompt': '<｜begin▁of▁sentence｜><｜User｜>你是资深的产业专家！请先浏览下述产业环节。\n氢能 氢气制备产业链环节。\n接下来，请你逐一浏览行业分类列表：[\'生产专用起重机制造\', \'信息系统集成服务\', \'锑矿采选\', \'金融信息服务\', \'维纶纤维制造\']，并在其中找出一眼看上去就属于氢能 氢气制备产业链环节的行业，不需要做过渡的行业内容挖掘。\n以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]<｜Assistant｜>',
 'predict': "<think>\n嗯，我现在需要解决这个问题，就是从给定的行业分类列表中找出属于氢能氢气制备产业链环节的行业。首先，我得先理解氢能和氢气制备产业链的环节是什么。氢能产业链通常包括氢气的生产、储存、运输、使用等环节，而氢气制备可能涉及氢气的合成、分离、纯化等技术。\n\n接下来，我看看提供的行业分类列表：['生产专用起重机制造', '信息系统集成服务', '锑矿采选', '金融信息服务', '维纶纤维制造']。现在我要逐一分析这些行业是否与氢能和氢气制备相关。\n\n第一个行业是生产专用起重机制造。起重机主要用于搬运和运输重物，虽然在工业生产中可能需要用到氢能设备，但起重机本身并不直接参与氢气的制备或相关流程，所以这个可能不太相关。\n\n第二个是信息系统集成服务。这个涉及信息技术，比如数据处理、网络系统集成等。氢能产业可能需要用到信息化系统来监控和管理氢气生产过程，但信息系统本身并不是氢气制备的直接环节，所以这个也不太相关。\n\n第三个是锑矿采选。锑是一种金属元素，主要应用于电子、电池等工业领域。锑矿采选主要涉及矿产的开采和选矿过程，和氢能制备没有直接关系，所以这个行业也不符合。\n\n第四个是金融信息服务。这个领域主要提供金融服务和技术服务，比如投资、银行、保险等。氢能产业可能需要金融支持，比如贷款或投资，但金融信息服务本身并不直接参与氢气的生产或制备，因此也不相关。\n\n第五个是维纶纤维制造。维纶纤维是一种合成纤维，主要用于制作绳索、纺织品等。虽然氢能可能在某些工业应用中使用到特定材料，但维纶纤维的生产与氢气制备没有直接联系，所以这个也不符合。\n\n综上所述，五个行业分类中，没有一个与氢能氢

In [ ]:
# industry_nodes[0] 

In [8]:
print(isinstance({1}, set))

True


In [9]:
def parse_item(data) -> Dict[str, List]:
    industry_nodes = []
    for item in data:
        industry_node = re.search(r"\n(.*?)产业链环节", item["prompt"])
        if industry_node:
            industry_node = industry_node.group(1)
        pred = re.search("</think>\n*(\[.*?\])", item["predict"], re.DOTALL)

        no_pred = 0
        if pred:
            pred = pred.group(1)
            try:
                pred = eval(pred)
                if len(pred) > 0 and isinstance(pred[0], str):
                    pred = pred
                else:
                    pred = []
            except:
                pred = []
        else:
            no_pred += 1
            pred = []
        industry_nodes.append([industry_node, pred])

    # 次数统计
    preds = {}
    for k, pred in industry_nodes:
        if k not in preds.keys():
            preds[k] = []
            
        preds[k].extend(pred)

    return preds

In [10]:
parse_item(data)

{'氢能 氢气制备': ['泵及真空设备制造',
  '能源矿产地质勘查',
  '其他专用化学产品制造',
  '其他专用设备制造',
  '水资源管理',
  '其他电子设备制造',
  '金冶炼',
  '电工仪器仪表制造',
  '货物运输代理',
  '其他技术推广服务',
  '阀门和旋塞制造',
  '其他化工产品批发',
  '工业设计服务',
  '电子电路制造',
  '药用辅料及包装材料制造',
  '快递服务',
  '炼油、化工生产专用设备制造',
  '半导体器件专用设备制造',
  '制浆和造纸专用设备制造',
  '金属密封件制造',
  '热电联产',
  '其他专用仪器制造',
  '电工机械专用设备制造',
  '镍钴冶炼',
  '新材料技术推广服务',
  '其他日用品生产专用设备制造',
  '远洋货物运输',
  '其他电子器件制造',
  '发电机及发电机组制造',
  '电子专用材料制造',
  '其他计算机制造',
  '金属工艺品制造',
  '低温仓储',
  '锅炉及辅助设备制造',
  '金属压力容器制造',
  '工程和技术研究和试验发展',
  '金属成形机床制造',
  '橡胶板、管、带制造',
  '其他未列明电气机械及器材制造',
  '陆地石油开采',
  '热力生产和供应',
  '其他电力生产',
  '其他电机制造',
  '其他道路货物运输',
  '运行维护服务',
  '其他非金属加工专用设备制造',
  '电子元器件与机电组件设备制造',
  '集装箱道路运输',
  '冶金专用设备制造',
  '其他稀有金属冶炼',
  '通用航空生产服务',
  '城市轨道交通',
  '供应用仪器仪表制造',
  '其他未列明制造业',
  '镁冶炼',
  '硅冶炼',
  '化学药品原料药制造',
  '管道和设备安装',
  '专项化学用品制造',
  '镍钴矿采选',
  '其他常用有色金属矿采选',
  '工程设计活动',
  '镍氢电池制造',
  '锂离子电池制造',
  '其他基础化学原料制造',
  '航天相关设备制造',
  '电车制造',
  '其他电池制造',
  '气体、液体分离及纯净设备制造',
  '管道工程建筑',
  '金属包装容器及材料制造',
  '日用

In [11]:
ans = {}
for file in os.listdir(p := "../infer/output/deepseek"):
    if not file.endswith(".jsonl"):
        continue

    file_name = os.path.join(p, file)
    print(file_name)
    data = load_jsonl(file_name)
    preds = parse_item(data)

    for k, industry_cls in preds.items():
        if k not in ans.keys():
            ans[k] = {}
        for item in industry_cls:
            # print(k, item)
            if item not in ans[k].keys():
                ans[k][item] = 0
            ans[k][item] += 1

../infer/output/deepseek/generated_predictions_deepseek3_0.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_1.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_2.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_3.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_4.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_5.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_6.jsonl
../infer/output/deepseek/generated_predictions_deepseek3_7.jsonl


In [12]:
ans

{'氢能 氢气制备': {'泵及真空设备制造': 7,
  '能源矿产地质勘查': 3,
  '其他专用化学产品制造': 5,
  '其他专用设备制造': 4,
  '水资源管理': 1,
  '其他电子设备制造': 5,
  '金冶炼': 1,
  '电工仪器仪表制造': 6,
  '货物运输代理': 3,
  '其他技术推广服务': 2,
  '阀门和旋塞制造': 3,
  '其他化工产品批发': 5,
  '工业设计服务': 1,
  '电子电路制造': 3,
  '药用辅料及包装材料制造': 1,
  '快递服务': 1,
  '炼油、化工生产专用设备制造': 4,
  '半导体器件专用设备制造': 2,
  '制浆和造纸专用设备制造': 1,
  '金属密封件制造': 7,
  '热电联产': 2,
  '其他专用仪器制造': 7,
  '电工机械专用设备制造': 7,
  '镍钴冶炼': 5,
  '新材料技术推广服务': 4,
  '其他日用品生产专用设备制造': 3,
  '远洋货物运输': 1,
  '其他电子器件制造': 3,
  '发电机及发电机组制造': 5,
  '电子专用材料制造': 4,
  '其他计算机制造': 1,
  '金属工艺品制造': 2,
  '低温仓储': 7,
  '锅炉及辅助设备制造': 7,
  '金属压力容器制造': 7,
  '工程和技术研究和试验发展': 8,
  '金属成形机床制造': 2,
  '橡胶板、管、带制造': 3,
  '其他未列明电气机械及器材制造': 4,
  '陆地石油开采': 1,
  '热力生产和供应': 2,
  '其他电力生产': 4,
  '其他电机制造': 2,
  '其他道路货物运输': 2,
  '运行维护服务': 2,
  '其他非金属加工专用设备制造': 6,
  '电子元器件与机电组件设备制造': 4,
  '集装箱道路运输': 2,
  '冶金专用设备制造': 3,
  '其他稀有金属冶炼': 1,
  '通用航空生产服务': 1,
  '城市轨道交通': 1,
  '供应用仪器仪表制造': 7,
  '其他未列明制造业': 2,
  '镁冶炼': 1,
  '硅冶炼': 1,
  '化学药品原料药制造': 4,
  '管道和设备安装': 6,
  '专项化学用品制造

## 输出

In [13]:
import pandas as pd

INDUSTRY_NAME = "产业链名"
INDUSTRY_NODE = "产业链环节"
INDUSTRY_DESC = "产业链环节描述"

df = pd.read_excel("../六链五群行业分类.xlsx")
df = df.rename(columns={"六链": INDUSTRY_NAME})

industry_cls_df = pd.read_excel("../小类注释_含一二三位数代码及名称.xlsx")
industry_cls_df = industry_cls_df[["小类代码2017", "小类名称2017"]]
industry_cls_df = industry_cls_df.rename(
    columns={"小类代码2017": "code", "小类名称2017": "name"}
)
industry_cls_df["code"] = industry_cls_df["code"].map(lambda x: str(x).zfill(4))
ALL_CLS_NAME = industry_cls_df["name"].values.tolist()

In [14]:
output_fold = "output/deepseek_7B"
for k, value in ans.items():
    del_cnt = 0
    tmp_data = []
    for kw, cnt in value.items():
        if kw not in ALL_CLS_NAME:
            del_cnt += 1
            continue
        tmp_data.append([kw, cnt])
    tmp_data.sort(key=lambda x: x[-1], reverse=True)

    tmp_df = pd.DataFrame(tmp_data, columns=["小类行业名", "投票数"])
    tmp_df.to_csv(os.path.join(output_fold, f"{k}.csv"))
    if del_cnt > 0:
        print(del_cnt, "个重要行业由于不在制定类别中，被移除")

1 个重要行业由于不在制定类别中，被移除
3 个重要行业由于不在制定类别中，被移除
2 个重要行业由于不在制定类别中，被移除
6 个重要行业由于不在制定类别中，被移除
6 个重要行业由于不在制定类别中，被移除
2 个重要行业由于不在制定类别中，被移除
1 个重要行业由于不在制定类别中，被移除
2 个重要行业由于不在制定类别中，被移除
3 个重要行业由于不在制定类别中，被移除
9 个重要行业由于不在制定类别中，被移除
6 个重要行业由于不在制定类别中，被移除
4 个重要行业由于不在制定类别中，被移除
5 个重要行业由于不在制定类别中，被移除
2 个重要行业由于不在制定类别中，被移除
5 个重要行业由于不在制定类别中，被移除
